# Przetwarzanie danych w językach R i Python - praca domowa 4

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<center><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form><c/enter>''')

*Szymon Adach *           
*Jacek Dziwulski*

24 maj 2017

In [43]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
py.sign_in('orgiele', '0hLF1CVCaH5sEU0CG9Ah')

# Tramwaje

Do analizy ruchu tramwajów w Warszawie wykorzystano dane zebrane przez naszego bota w niedzielę 14. maja oraz poniedziałek 15. maja. Wybrano te dwa dni, aby możliwe było sprawdzenie różnic w kursowaniu tramwajów w weekend oraz dzień powszedni. Ponadto analizie poddano dane z dwóch dni marca 2016 roku, dostarczone przez prowadzącego.

Dane zbierane były z dwóch endpointów API udostępnianego przez ZTM. Zaletą pierwszego endpointu był kilkukrotnie mniejszy czas między aktualizacją danych o położeniu, z kolei drugi posiadał informację logiczną o tym, czy dany tramwaj jest niskopodłogowy.

Dane po pobraniu zostały poddane obróbce, która usunęła z ramek danych pojazdy o  niepoprawnej geolokacji lub błędnej dacie.

## Sieć tramwajowa

<img src="data\15-05\trams-15-filtered.png"/>
<center style="padding-top:10px">Rysunek 3: Sieć tramwajowa Warszawy, na podstawie danych z 15.05.2017 (dzień powszedni).</center>

Po pierwsze, możemy zauważyć, że przed ostatni rok sieć tramwajowa nie uległa znączącym zmianom. Pojawiły się pewne nowe linie, inne mogły zostać wstrzymane, lecz obszar po którym się poruszają jest mniej więcej taki sam.

Widoczna jest asymetria sieci tramwajowej - jest ona znacznie gęstsza w lewobrzeżnej części miasta. Oczywistym powodem jest trudność w "przeprawie" tramwajów przez Wisłę. Aktualnie realizowana jest ona przez następujące mosty:

<div class="datagrid">
<table>
<thead>
    <tr><th>Nazwa mostu</th><th>Liczba torów tramwajowych</th>
</thead>
<tbody>
    <tr><td>Most Poniatowskiego</td><td>2</td></tr>
    <tr><td>Most Gdański</td><td>2</td></tr>
    <tr><td>Most Śląsko-Dąbrowski</td><td>2N → 2</td></tr>    
    <tr><td>Most Świętokrzyski</td><td>0(2)</td></tr>
    <tr><td>Most Północny</td><td>(2)</td></tr>
</tbody>
</table>
</div>

<center style="padding-top:10px"> Tabela 1: Liczba torów tramwajowych na mostach w Warszawie </center>

* W nawiasach przedstawiono ilość planowanych pasów ruchu.
* Strzałkami oznaczono kolejne zmiany ilości pasów.
* Literą N oznaczono pasy niewydzielone, tj. tory tramwajowe niewydzielone z jezdni lub drogi rowerowe niewydzielone z chodnika.

Dane za: [Wikipedia](https://pl.wikipedia.org/wiki/Mosty_w_Warszawie)

Powyższe mapy można skonfrontować z dostępną na Wikipedii mapą sieci tramwajowej stolicy sprzed roku 1939:
<img src="historical_tram_map.png" width=50% />

<center> Rysunek 4: Mapa sieci tramwajowej w Warszawie sprzed 1939 roku </center>


## Punkty krytyczne sieci tramwajowej

Do wyznaczenia krytycznych punktów sieci tramwajowej posłużyliśmy się heatmapami:

In [16]:
from IPython.display import HTML, display
display(HTML(
    "<table><tr><td><img src='data/14-05/trams-heatmap.png'/>"+
    "</td><td><img src='data/15-05/trams-heatmap.png'/></td></tr></table>"))

,


<center>Rysunek 5: Po lewej heatmapa dla 14.05.2017 (niedziela), po prawej dla 15.05.2017 (poniedziałek)</center>

Krytycznymi punktami są z pewnością przystanki Dworzec Centralny, Centrum oraz przystanki w ich okolicy.  
Zauważyć należy również stosunkowo zwiększone natężenie ruchu na Ochocie, w okolicy skrzyżowania ulic Bitwy Warszawskiej 1920r. oraz Banacha, gdzie spotykają się linie obsługiwane przez zajezdnię na Okęciu oraz zajezdnię przy Banacha.   Natężenie stopniowo spada aż do Placu Zawiszy, gdzie część tramwajów jedzie na wschód w kierunku Centrum, a pozostałe na północ.  
Innym ważnym punktem na tramwajowej mapie Warszawy jest Rondo "Radosława", gdzie znajduje się istotny punkt przesiadkowy w stronę Pragi i regularnie kursuje aż 8 linii tramwajowych.
Po lewej stronie Wisły najistotniejsze wydają się przystanki przy Stadionie Narodowym.
Oczywistym faktem jest, że szczególnie wyróżniają się na mapach zajezdnie tramwajowe.

Jeśli chodzi o różnice pomiędzy heatmapą z niedzieli a heatmapą z poniedziałku, to (poza ogólnie większym natężeniem ruchu) zaobserwować można zwiększenie znaczenia zajezdni Mokotów przy skrzyżowaniu Woronicza i Wołoskiej, a także istotne zwiększenie częstości kursowania w kierunku Żoliborza i Bielan.

Powyższe heatmapy możemy porównać z danymi z 23.03.2016.
<img style="padding-top:10px" src='data\2016-03-23\trams-heatmap.png'/>
<center style="padding-top:10px">Rysunek 6: Heatmapa ruchu tramwajów 23.03.2016.</center>

Można zaobserwować zwiększone natężenie w stosunku do roku 2017. Widoczne jest to między innymi w okoliach Ronda Daszyńskiego oraz Dworca Wileńskiego. Wydawałoby się, że otwarcie w 2015 roku linii metra M2 łączącej te dwa przystanki, powinno zniwelować ruch tramwajowy w tych okolicach. Być może ZTM stopniowo zmieniał przebieg i nateżenie kursowania linii tam kursująych - stąd zdecydowanie niższe natężenie ruchu w maju 2017.

## Czasy dojazdu

In [19]:
times = pd.read_csv('data/15-05/trams3-filtered/time-data.csv')
times = times.reindex_axis(times.mean().sort_values().index, axis=1)
means = times.mean()
stds = times.std()

timetable_times = [21, 17, 16, 20, 29, 29]

trace1 = go.Bar(
        x = list(pd.DataFrame(means).index),
        y = means,
        name = "Średni czas przejazdu",
        error_y=dict(
            type='data',
            array=stds,
            visible=True
        )
    )

trace2 = go.Bar(
        x = list(pd.DataFrame(means).index),
        name = "Czas przejazdu wg ZTM",
        y = timetable_times,
    )

data = [trace1, trace2]
layout = go.Layout(
    title='Czas dojazdu tramwajem na Dworzec Centralny z wybranych dzielnic w godzinach szczytu',
    barmode='group',
        xaxis=dict(
        title='Dzielnica'
    ),
    yaxis=dict(
        title='Czas [min]'
    )
)
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')


<center style="padding-top:10px">Rysunek 7: Wykres zależności czasu dojazdu od dzielnicy, poniedziałek 15.05.2017</center>

W celu utworzenia powyższego wykresu, rozważyliśmy najważniejsze przystanki z każdej dzielnicy i wybraliśmy dla każdej dzielnicy po jednym - takim, aby w przybliżeniu odegłości od przystanku docelowego były równe. Za docelowy przystanek uznaliśmy Dworzec Centralny. Z wyborem tym można polemizować, gdyż w większości powyższych dzielnic znajdują się punkty przesiadkowe, np. stacje kolei miejskiej lub metra, jednak nasz wybór znacząco ułatwia analizę.

Zaobserwować można rozbieżności rzędu 2-3 minut między rozkładowymi czasami przejazdów, a czasami zmierzonymi. Ciekawym przypadkiem jest Ochota, dla której odchylenie standardowe osiąga wartość ponad 6 minut, co stanowi ponad 30% całkowitego czasu przejazdu. Prawdopodobnie wiąże się to z trasą linii kursujących na tym odcinku - muszą one przeciąć dwa ruchliwe węzły w okolicy ul. Bitwy Warszawskiej 1920 r. i Placu Zawiszy. W pesymistycznym przypadku cykle sygnalizacji drogowej mogą tam mieć stosunkowo duży wpływ na ewentualne opóźnienia.

Należy także pamiętać, że wykres ten nie prezentuje optymalnego czasu dojazdu, gdyż często wykorzystanie **tylko** tramwaju jest wolniejsze niż np. podróż metrem.

Dane z roku 2016 dostarczone przez prowadzącego są pobrane z endpointu api, w którego dokumentacji możemy przeczytać "Dane są aktualizowane co 30 sekund.". W związku z tym analiza spóźnień byłaby bardzo niedokładna, gdyż opiera się ona na ustalaniu różnic między timestampami na wybranych przystankach (z odpowiednią dokładnością co do pozycji na przystanku).

## Tramwaje niskopodłogowe

In [20]:
df = pd.read_csv("data/23-05/tram-counts.csv")

total_count = df['All']
lowfloor = df['Lowfloor']
hours = [7, 8, 9, 10, 11, 12]
    
trace1 = {
    'y': lowfloor,
    'x': hours,
    'name': 'Liczba niskopodłogowych',
    'type': 'bar'
}
trace2 = {
    'y': total_count,
    'x': hours,
    'name':'Całkowita liczba przejazdów',
    'type': 'bar'
}

data = [trace1, trace2]
layout = {
    'title': 'Liczba przejazdów i liczba przejazdów tramwajami niskopodłogowymi we wszystkich przejazdach w ciągu danej godziny',
    'yaxis': {'title':'Liczba niskopodłogowych'},
    'barmode': 'relative'
}
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

<center style="padding-top:10px">Rysunek 8: Całkowita liczba przejazdów podczas danej godziny oraz liczba przejazdów realizowanych składami niskopodłogowymi w tej godzinie</center>

In [24]:
from IPython.display import display, HTML
df = pd.DataFrame({'Godzina': hours, 'Odsetek niskopodłogowych': round(lowfloor / total_count, 3) * 100})
HTML(df.to_html(index=False, classes=['table']))

Godzina,Odsetek niskopodłogowych
7,55.3
8,55.4
9,57.1
10,62.7
11,65.7
12,65.7


<center>Tabela 2: Odsetek tramwajów niskopodłogowych w zależności od godziny.</center>

Zgodnie z przewidywaniami, liczba przejazdów osiąga wartość maksymalną dla zakresu godzin 8-9, czyli godzin szczytu, kiedy młodzież jeździ do szkół, a dorośli do pracy. W ruchu utrzymywana jest w przybliżeniu podobna liczba pojazdów niskopodłogowych, zatem ich odsetek rośnie, kiedy całkowita liczba przejazdów maleje - z tras usuwane są starsze pojazdy.

In [25]:
df = pd.read_csv("data/2016-03-23/tram-counts.csv")

total_count = df['All']
lowfloor = df['Lowfloor']
hours = [7, 8, 9, 10, 11, 12]
    
trace1 = {
    'y': lowfloor,
    'x': hours,
    'name': 'Liczba niskopodłogowych',
    'type': 'bar'
}
trace2 = {
    'y': total_count,
    'x': hours,
    'name':'Całkowita liczba przejazdów',
    'type': 'bar'
}

data = [trace1, trace2]
layout = {
    'title': 'Liczba przejazdów i liczba przejazdów tramwajami niskopodłogowymi we wszystkich przejazdach <br> w ciągu danej godziny w 2016 roku',
    'yaxis': {'title':'Liczba niskopodłogowych'},
    'barmode': 'relative'
}
fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

<center> Rysunek 9: Całkowita liczba przejazdów podczas danej godziny oraz liczba przejazdów realizowanych składami niskopodłogowymi w tej godzinie w 2016 roku. </center>

In [26]:
from IPython.display import display, HTML
df = pd.DataFrame({'Godzina': hours, 'Odsetek niskopodłogowych': round(lowfloor / total_count, 3) * 100})
HTML(df.to_html(index=False, classes=['table']))

Godzina,Odsetek niskopodłogowych
7,47.7
8,47.7
9,49.2
10,55.3
11,56.1
12,56.5


<center>Tabela 3: Odsetek tramwajów niskopodłogowych w zależności od godziny w 2016 roku.</center>

Z powyższych danych możemy wywnioskować, że zwiększył się procent tramawajów niskopodłogowych w ruchu, nawet o 10 punktów procentowych. W wykresów możemy odczytać również, że zwiększenie liczby pojazdów niskopodłogowych nie koniecznie spowodowało zmniejszenie liczności innych pojazdów, ponieważ całkowita liczba wszystkich pojazdów jest większa teraz, niż była w 2016 roku.

In [27]:
df = pd.read_csv("data/15-05/tram-counts.csv")

trace1 = go.Scatter(
    x=df["Hour"][8:],
    y=df["Count"][8:],
    name='Liczba przejazdów (2017)',
    line = dict(dash = 'dot'),
)

df2 = pd.read_csv("data/2016-03-23/tram-counts2.csv")

trace2 = go.Scatter(
    x=df2["Hour"][8:],
    y=df2["Count"][8:],
    name='Liczba przejazdów (2016)',
    line = dict(dash = 'dot'),
)

data = [trace1, trace2]
layout = go.Layout(
    title='Liczba przejazdów w ciągu godziny',
    yaxis=dict(
        title='Liczba przejazdów'
    ),
    xaxis=dict(
        title='Godzina'
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic')

<center style="padding-top:10px">Rysunek 12: Całkowita liczba przejazdów w ciągu godziny</center>

Widoczne są godziny szczytu, podczas których ZTM udostępnia nawet o 35% więcej tramwajów niż podczas dziennego minimum o godzinie 11 (minimum wyznaczone po odrzuceniu godzin po 21).

W 2016 roku rozkład tramwajów w ruchu wyglądał bardzo podobnie - jedyna istotna różnica to mniejsza ilość tramwajów w ruchu. Wg informacji prasowych ZTM, na mniej uczęszczanych trasach (np. linia 6 Gocławek-Młociny) pojedyncze składy dwuwagonowe rozbito na dwa składy, dzięki czemu możliwe jest częstsze kursowanie tych tramwajów. Z tego może wynikać znacząco większa liczba przejazdów w 2017 roku, mimo braku zakupów nowych składów.


# Autobusy

Autobusy możemy podzielić na pewne kategorie, które później posłużą do analizy ich ruchu. Te kategorie to:

* linie zwykłe (102 - 262)
* linie zwykłe okresowe (300 - 397)
* linie przyspieszone (500 - 527)
* linie przyspieszone okresowe (401 - 414)
* linie strefowe (700 - 743)
* linie strefowe uzupełniające (L-1 - L40)
* linie strefowe okresowe (800)
* linie ekspresowe (E-1 - E-9)
* linie nocne (N01 - N95)
* linie specjalne (900)


## Linie zwykłe

In [38]:
display(HTML(
    "<table><tr><td><img src='data/14-05/buses-normal14-filtered2.png'/>"+
    "</td><td><img src='data/15-05/buses-normal15-filtered.png'/></td></tr></table>"))

,


<center style="padding-top:10px">Rysunek x: Trasa przebyta przez autobusy zwykłe 14 maja 2017 roku (po lewej), oraz 15 maja 2017 roku (po prawej).</center>

In [29]:
distance = pd.concat([pd.read_json('file:data/14-05/busesNormalCount.json'), 
                      pd.read_json('file:data/15-05/busesNormalCount.json')])[0].tolist()
distance = pd.DataFrame({'Dzień':['14.05.2017', '15.05.2017'], 'Liczba autobusów':distance}, 
                        columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,566
15.05.2017,935


<center style="padding-top:10px">Tabela x: Liczba autobusów linii zwykłych w ruchu w wybranych dniach. </center>

## Linie przyspieszone

In [30]:
display(HTML(
    "<table><tr><td><img src='data/14-05/buses-fast14-filtered.png'/>"+
    "</td><td><img src='data/15-05/buses-fast15-filtered.png'/></td></tr></table>"))

,


<center> Rysunek x: Trasa przebyta przez autobusy przespieszone 14 maja 2017 roku (po lewej) oraz 15 maja 2017 roku (po prawej). </center>

In [31]:
distance = pd.concat([pd.read_json('file:data/14-05/busesFastCount.json'), 
                      pd.read_json('file:data/15-05/busesFastCount.json')])[0].tolist()
distance = pd.DataFrame({'Dzień':['14.05.2017', '15.05.2017'], 'Liczba autobusów':distance}, 
                        columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,142
15.05.2017,274


<center>Tabela x: Liczba autobusów linii przyspieszonych w ruchu w wybranych dniach. </center>

## Linie nocne

<img src="data\14-05\buses-night14-filtered.png" />

<center style="padding-top:10px">Rysunek x: Trasa przebyta przez autobusy nocne 14 maja 2017 roku. </center>

In [32]:
distance = pd.read_json('file:data/14-05/busesNightCount.json')[0].tolist()
distance = pd.DataFrame({'Dzień':'14.05.2017', 'Liczba autobusów':distance}, columns=['Dzień', 'Liczba autobusów'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Liczba autobusów
14.05.2017,151


<center style="padding-top:10px">Tabela x: Liczba autobusów linii nocnych w ruchu w wybranych dniach. </center>

## Pokonany dystans

In [33]:
distance = pd.concat([pd.read_json('file:data/14-05/busDistance-night.json'),
                     pd.read_json('file:data/14-05/busDistance-day.json'),
                     pd.read_json('file:data/15-05/busDistance-day.json')])

distanceList = [x/1000 for x in distance[0].tolist()]

index = ['14.05.2017 - noc', '14.05.2017 - dzień (weekend)', '15.05.2017 - dzień']
trace1 = go.Bar(
        x = index,
        y = distanceList,
        name = "Pokonany dystans"
    )
layout = go.Layout(
    title='Całkowity dystans w kilometrach pokonany przez autobusy w zależności od dnia',
    barmode='group',
    xaxis=dict(
        title='Dzień'
    ),
    yaxis=dict(
        title='Dystans [km]'
    )
)

data = [trace1]

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

<center> Rysunek x: Wykres pokonanego dystansu przez autobusy w zależności od dnia </center>

In [34]:
distance = pd.DataFrame({'Dzień':index, 'Dystans [km]':[round(dist) for dist in distanceList]}, columns=['Dzień', 'Dystans [km]'])
HTML(distance.to_html(classes=['table'], index = False))

Dzień,Dystans [km]
14.05.2017 - noc,49676
14.05.2017 - dzień (weekend),134409
15.05.2017 - dzień,337064


<center> Tabela x: Całkowity dystans w kilometrach, pokonany przez autobusy w wybrane dni. </center>

Wiemy jaki dystans pokonują autobusy. Możemy też, w przybliżeniu, określić średnie spalanie pojazdów. Najpopularniejszy autobus w Warszawie - Solaris Urbino 18 - według różnych doniesień, spala od 40 do 45 litrów na 100 kilometrów.

Nie wiemy, jakie są koszty zakupu paliwa. Ponieważ są to ilości hurtowe (co za chwilę się okaże), możemy założyć, że koszt jest znacznie niższy, niż dla normalnego człowieka na stacji benzynowej. Przyjmijmy 3 złote na litr benzyny. Następnie, dla 15.05.2017, możemy dokonać następujących obliczeń:
                    

<center>337063 \* 40/100 = 134825.2 litrów paliwa **dziennie**, co przekłada się na          
134825.2 \* 3 = **404475.6 złotych**</center>

Analogicznie dla 14.05.2017 w nocy, jest to 59610 złotych, oraz w dzień 161290.8 złotych dziennie. 

Obliczony koszt możemy przedstawić jeszcze w inny sposób - najtańszy bilet normalny, 20-minutowy, kosztuje w Warszawie 3.40 złotych. Aby zwrócił się oszacowany koszt utrzymania w dzień powszedni, należy sprzedać conajmniej 118964 takich biletów, w ciągu jednego dnia.

## Średnia prędkość

In [35]:
day14speed = pd.concat([pd.read_json('file:data/14-05/busSpeed-day-grouped.json'),
                     pd.read_json('file:data/14-05/busSpeed-night-grouped.json')])

day15speed = pd.read_json('file:data/15-05/busSpeed-day-grouped.json')

trace1 = go.Bar(
        x = day14speed['Type'],
        y = day14speed['V'],
        name = "14.05.2017"
    )

trace2 = go.Bar(
        x = day15speed['Type'],
        y = day15speed['V'],
        name = "15.05.2017"
    )

layout = go.Layout(
    title='Średnia prędkość wybranych linii autobusowych',
    barmode='group',
    xaxis=dict(
        title='Rodzaj linii'
    ),
    yaxis=dict(
        title='Prędkość [km\h]'
    )
)

data = [trace1, trace2]

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename='basic')

<center> Rysunek x: Średnia prędkość wybranych linii autobusowych w zależności od dnia </center>

In [36]:
day15speedModified = pd.concat([day15speed, pd.DataFrame({'Type':'Nocne', 'V':'-'}, index = [0])])

speed =  pd.DataFrame({ 'Rodzaj linii':day14speed['Type'], '14.05.2017':day14speed['V'],
                      '15.05.2017':day15speedModified['V']}, columns=['Rodzaj linii', '14.05.2017', '15.05.2017'])
HTML(speed.to_html(classes=['table'], index=False))

Rodzaj linii,14.05.2017,15.05.2017
Zwykle,26.4748,25.1964
Strefowe,31.4119,28.7053
Przyspieszone,27.0149,24.5094
Przyspieszone okresowe,27.1122,25.1269
Nocne,30.2197,-


<center>Tabela x: Średnia prędkość w km/h wybranych linii autobusowych w zależności od dnia </center>